# Operation Wocao Activity
Detects activity mentioned in Operation Wocao report

## Rule Content
```
- action: global
  title: Operation Wocao Activity
  id: 74ad4314-482e-4c3e-b237-3f7ed3b9ca8d
  author: Florian Roth
  status: experimental
  description: Detects activity mentioned in Operation Wocao report
  references:
  - https://www.fox-it.com/en/news/whitepapers/operation-wocao-shining-a-light-on-one-of-chinas-hidden-hacking-groups/
  - https://twitter.com/SBousseaden/status/1207671369963646976
  date: 2019/12/20
  falsepositives:
  - Administrators that use checkadmin.exe tool to enumerate local administrators
  level: high
- logsource:
    product: windows
    service: security
  detection:
    selection:
      EventID: 4799
      GroupName: Administrators
      ProcessName: '*\checkadmin.exe'
    condition: selection
- logsource:
    category: process_creation
    product: windows
  detection:
    selection:
      CommandLine|contains:
      - checkadmin.exe 127.0.0.1 -all
      - netsh advfirewall firewall add rule name=powershell dir=in
      - cmd /c powershell.exe -ep bypass -file c:\s.ps1
      - /tn win32times /f
      - create win32times binPath=
      - \c$\windows\system32\devmgr.dll
      - ' -exec bypass -enc JgAg'
      - type *keepass\KeePass.config.xml
      - iie.exe iie.txt
      - reg query HKEY_CURRENT_USER\Software\*\PuTTY\Sessions\
    condition: selection

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"4799" AND group_name:"Administrators" AND process_path.keyword:*\\checkadmin.exe)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='process_command_line.keyword:(*checkadmin.exe\ 127.0.0.1\ \-all* OR *netsh\ advfirewall\ firewall\ add\ rule\ name\=powershell\ dir\=in* OR *cmd\ \/c\ powershell.exe\ \-ep\ bypass\ \-file\ c\:\\s.ps1* OR *\/tn\ win32times\ \/f* OR *create\ win32times\ binPath\=* OR *\\c$\\windows\\system32\\devmgr.dll* OR *\ \-exec\ bypass\ \-enc\ JgAg* OR *type\ *keepass\\KeePass.config.xml* OR *iie.exe\ iie.txt* OR *reg\ query\ HKEY_CURRENT_USER\\Software\*\\PuTTY\\Sessions\*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()